In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('t20_master.csv')

C:\Users\manis\AppData\Local\Temp\ipykernel_1312\714940151.py:1: DtypeWarning: Columns (1,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('t20_master.csv')


In [3]:
df.head()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,1001349,2016/17,2017-02-17,Melbourne Cricket Ground,1,0.1,Australia,Sri Lanka,AJ Finch,M Klinger,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001349,2016/17,2017-02-17,Melbourne Cricket Ground,1,0.2,Australia,Sri Lanka,AJ Finch,M Klinger,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1001349,2016/17,2017-02-17,Melbourne Cricket Ground,1,0.3,Australia,Sri Lanka,AJ Finch,M Klinger,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1001349,2016/17,2017-02-17,Melbourne Cricket Ground,1,0.4,Australia,Sri Lanka,M Klinger,AJ Finch,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1001349,2016/17,2017-02-17,Melbourne Cricket Ground,1,0.5,Australia,Sri Lanka,M Klinger,AJ Finch,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(1448859, 22)

In [6]:
df.dtypes

match_id                    int64
season                     object
start_date                 object
venue                      object
innings                     int64
ball                      float64
batting_team               object
bowling_team               object
striker                    object
non_striker                object
bowler                     object
runs_off_bat                int64
extras                      int64
wides                     float64
noballs                   float64
byes                      float64
legbyes                   float64
penalty                   float64
wicket_type                object
player_dismissed           object
other_wicket_type          object
other_player_dismissed     object
dtype: object

In [7]:
print(df.iloc[:, [1, 20, 21]].head())

    season other_wicket_type other_player_dismissed
0  2016/17               NaN                    NaN
1  2016/17               NaN                    NaN
2  2016/17               NaN                    NaN
3  2016/17               NaN                    NaN
4  2016/17               NaN                    NaN


In [9]:
for col in [1, 20, 21]:
    print(f"Column {col}:")
    print(df.iloc[:, col].apply(type).value_counts())
    print("\n")

Column 1:
<class 'str'>    1022875
<class 'int'>     425984
Name: season, dtype: int64


Column 20:
<class 'float'>    1448857
<class 'str'>            2
Name: other_wicket_type, dtype: int64


Column 21:
<class 'float'>    1448857
<class 'str'>            2
Name: other_player_dismissed, dtype: int64




In [10]:
striker_match_counts = df.groupby(['striker', 'match_id']).size().reset_index(name='counts')
total_matches_by_striker = striker_match_counts.groupby('striker')['match_id'].nunique().reset_index(name='total_matches')

In [11]:
total_matches_by_striker.head()

,striker,total_matches
0,A Ahmadhel,3
1,A Amado,2
2,A Andrews,1
3,A Ashish Reddy,23
4,A Ashok,12


In [12]:
total_runs_by_striker = df.groupby('striker')['runs_off_bat'].sum().reset_index(name='total_runs')

In [13]:
total_runs_by_striker.head()

,striker,total_runs
0,A Ahmadhel,6
1,A Amado,19
2,A Andrews,4
3,A Ashish Reddy,280
4,A Ashok,34


In [14]:
balls_faced_by_striker = df['striker'].value_counts().reset_index(name='balls_faced')
balls_faced_by_striker.columns = ['striker', 'balls_faced']

In [15]:
balls_faced_by_striker.head()

,striker,balls_faced
0,CH Gayle,9604
1,V Kohli,8732
2,AJ Finch,7846
3,RG Sharma,7731
4,DA Warner,7410


In [18]:
df['isPlayerOut'] = 0
df['isPlayerOut'] = np.where(df['wicket_type'].notna(), 1, 0)

In [20]:
not_out_strikers = df[(df['isPlayerOut'] == 0) & (df.groupby(['match_id', 'innings'])['ball'].transform(max) == df['ball'])]
not_out_counts_strikers = not_out_strikers['striker'].value_counts().reset_index(name='not_out_count')
not_out_counts_strikers.columns = ['player', 'not_outs_as_striker']

not_out_non_strikers = df[(df.groupby(['match_id', 'innings'])['ball'].transform(max) == df['ball'])]
not_out_counts_non_strikers = not_out_non_strikers['non_striker'].value_counts().reset_index(name='not_out_count')
not_out_counts_non_strikers.columns = ['player', 'not_outs_as_non_striker']

In [21]:
total_not_outs = pd.merge(not_out_counts_strikers, not_out_counts_non_strikers, on='player', how='outer')
total_not_outs.fillna(0, inplace=True)
total_not_outs['total_not_outs'] = total_not_outs['not_outs_as_striker'] + total_not_outs['not_outs_as_non_striker']

In [22]:
total_not_outs.head()

,player,not_outs_as_striker,not_outs_as_non_striker,total_not_outs
0,MS Dhoni,84.0,41.0,125.0
1,KA Pollard,83.0,50.0,133.0
2,DA Miller,61.0,52.0,113.0
3,AD Russell,59.0,32.0,91.0
4,DJ Bravo,51.0,56.0,107.0


In [23]:
total_not_outs.rename(columns={'player': 'striker'}, inplace=True)

striker_stats = pd.merge(total_runs_by_striker, total_matches_by_striker, on='striker', how='left')
striker_stats = pd.merge(striker_stats, total_not_outs[['striker', 'total_not_outs']], on='striker', how='left')

striker_stats['total_not_outs'] = striker_stats['total_not_outs'].fillna(0)

striker_stats['times_out'] = striker_stats['total_matches'] - striker_stats['total_not_outs']
striker_stats['average'] = striker_stats.apply(
    lambda row: row['total_runs'] / row['times_out'] if row['times_out'] > 0 else 0, axis=1)

striker_stats['dismissed'] = striker_stats['times_out']

averages = striker_stats[['striker', 'total_runs', 'dismissed', 'average']]

In [25]:
averages['average'] = averages['average'].round(2)
averages.head()

C:\Users\manis\AppData\Local\Temp\ipykernel_1312\2595679526.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  averages['average'] = averages['average'].round(2)


,striker,total_runs,dismissed,average
0,A Ahmadhel,6,3.0,2.00
1,A Amado,19,2.0,9.50
2,A Andrews,4,0.0,0.00
3,A Ashish Reddy,280,15.0,18.67
4,A Ashok,34,5.0,6.80


In [26]:
player_name = 'MS Dhoni'
player_average = averages.loc[averages['striker'] == player_name, 'average'].values[0]

print(f"The batting average for {player_name} is: {player_average}")

The batting average for MS Dhoni is: 38.06


In [27]:
runs_column = 'total_runs'
balls_faced_column = 'balls_faced'

player_stats = pd.merge(total_runs_by_striker, balls_faced_by_striker, on='striker', how='left')
player_stats['strike_rate'] = (player_stats[runs_column] / player_stats[balls_faced_column]) * 100
player_stats['strike_rate'].replace([np.inf, -np.inf], np.nan, inplace=True)

player_stats['strike_rate'] = player_stats['strike_rate'].fillna(0)
strike_rate_df = player_stats[['striker', 'strike_rate']].copy()

In [29]:
strike_rate_df['strike_rate'] = strike_rate_df['strike_rate'].round(2)
strike_rate_df.head()

,striker,strike_rate
0,A Ahmadhel,28.57
1,A Amado,67.86
2,A Andrews,66.67
3,A Ashish Reddy,142.86
4,A Ashok,87.18


In [31]:
runs_in_innings = df.groupby(['match_id', 'innings', 'striker'])['runs_off_bat'].sum().reset_index()
centuries = runs_in_innings[runs_in_innings['runs_off_bat'] >= 100]

centuries_count = centuries['striker'].value_counts().reset_index()
centuries_count.columns = ['striker', 'centuries']

player_stats_with_centuries = pd.merge(player_stats[['striker']], centuries_count, on='striker', how='left')
player_stats_with_centuries['centuries'] = player_stats_with_centuries['centuries'].fillna(0)

final_df = player_stats_with_centuries[['striker', 'centuries']].copy()

In [32]:
final_df.head()

,striker,centuries
0,A Ahmadhel,0.0
1,A Amado,0.0
2,A Andrews,0.0
3,A Ashish Reddy,0.0
4,A Ashok,0.0


In [34]:
top_players = final_df.sort_values(by='centuries', ascending=False)
top_10_centuries = top_players.head(10)

top_10_centuries

,striker,centuries
854,CH Gayle,18.0
4319,V Kohli,8.0
195,AJ Finch,8.0
2443,M Klinger,7.0
3820,SR Watson,6.0
729,Babar Azam,6.0
163,AD Hales,6.0
2358,LJ Wright,6.0
2180,KL Rahul,6.0
1029,DA Warner,6.0


In [35]:
runs_in_innings = df.groupby(['match_id', 'innings', 'striker'])['runs_off_bat'].sum().reset_index()
half_centuries = runs_in_innings[(runs_in_innings['runs_off_bat'] >= 50) & (runs_in_innings['runs_off_bat'] < 100)]

half_centuries_count = half_centuries['striker'].value_counts().reset_index()
half_centuries_count.columns = ['striker', 'half_centuries']

player_stats_with_half_centuries = pd.merge(player_stats[['striker']], half_centuries_count, on='striker', how='left')
player_stats_with_half_centuries['half_centuries'] = player_stats_with_half_centuries['half_centuries'].fillna(0)

final_df_half_centuries = player_stats_with_half_centuries[['striker', 'half_centuries']].copy()

In [36]:
top_half_century_scorers = final_df_half_centuries.sort_values(by='half_centuries', ascending=False)
top_10_half_centuries = top_half_century_scorers.head(10)

top_10_half_centuries

,striker,half_centuries
1029,DA Warner,90.0
4319,V Kohli,87.0
854,CH Gayle,79.0
195,AJ Finch,71.0
3378,RG Sharma,71.0
148,AB de Villiers,64.0
729,Babar Azam,63.0
1844,JC Buttler,61.0
3575,S Dhawan,61.0
163,AD Hales,58.0


In [37]:
runs_in_innings = df.groupby(['match_id', 'innings', 'striker'])['runs_off_bat'].sum().reset_index()

thirties = runs_in_innings[(runs_in_innings['runs_off_bat'] >= 30) & (runs_in_innings['runs_off_bat'] < 50)]
thirties_count = thirties['striker'].value_counts().reset_index()
thirties_count.columns = ['striker', 'thirties']

player_stats_with_thirties = pd.merge(player_stats[['striker']], thirties_count, on='striker', how='left')
player_stats_with_thirties['thirties'] = player_stats_with_thirties['thirties'].fillna(0)
final_df_thirties = player_stats_with_thirties[['striker', 'thirties']].copy()

In [38]:
top_thirty_scorers = final_df_thirties.sort_values(by='thirties', ascending=False)
top_10_thirties = top_thirty_scorers.head(10)


top_10_thirties

,striker,thirties
2136,KA Pollard,78.0
854,CH Gayle,73.0
195,AJ Finch,72.0
3983,Shoaib Malik,71.0
787,C Munro,67.0
641,BB McCullum,66.0
4319,V Kohli,64.0
163,AD Hales,64.0
1026,DA Miller,62.0
3378,RG Sharma,61.0


In [39]:
runs_filter = df['runs_off_bat'].isin([0, 1, 2, 3, 4, 6])

runs_distribution = pd.crosstab(df[runs_filter]['striker'], df[runs_filter]['runs_off_bat'])

runs_distribution.columns = ['0s', '1s', '2s', '3s', '4s', '6s']
runs_distribution.reset_index(inplace=True)

In [40]:
runs_distribution.head()

,striker,0s,1s,2s,3s,4s,6s
0,A Ahmadhel,19,0,1,0,1,0
1,A Amado,16,9,1,0,2,0
2,A Andrews,5,0,0,0,1,0
3,A Ashish Reddy,61,83,20,1,16,15
4,A Ashok,18,14,4,0,3,0


In [41]:
from functools import reduce

In [42]:
dataframes = [
    total_matches_by_striker,
    total_runs_by_striker,
    balls_faced_by_striker,
    total_not_outs,
    averages,
    strike_rate_df,
    final_df,
    final_df_half_centuries,
    final_df_thirties,
    runs_distribution
]

In [43]:
final_combined_df = reduce(lambda left, right: pd.merge(left, right, on='striker', how='outer'), dataframes)
final_combined_df.fillna(0, inplace=True)

final_combined_df.to_csv('batsman_stats.csv', index=False)

In [44]:
final_combined_df.head()

,striker,total_matches,total_runs_x,balls_faced,not_outs_as_striker,not_outs_as_non_striker,total_not_outs,total_runs_y,dismissed,average,strike_rate,centuries,half_centuries,thirties,0s,1s,2s,3s,4s,6s
0,A Ahmadhel,3.0,6.0,21.0,0.0,0.0,0.0,6.0,3.0,2.00,28.57,0.0,0.0,0.0,19.0,0.0,1.0,0.0,1.0,0.0
1,A Amado,2.0,19.0,28.0,0.0,0.0,0.0,19.0,2.0,9.50,67.86,0.0,0.0,0.0,16.0,9.0,1.0,0.0,2.0,0.0
2,A Andrews,1.0,4.0,6.0,0.0,1.0,1.0,4.0,0.0,0.00,66.67,0.0,0.0,0.0,5.0,0.0,0.0,0.0,1.0,0.0
3,A Ashish Reddy,23.0,280.0,196.0,4.0,4.0,8.0,280.0,15.0,18.67,142.86,0.0,0.0,2.0,61.0,83.0,20.0,1.0,16.0,15.0
4,A Ashok,12.0,34.0,39.0,3.0,4.0,7.0,34.0,5.0,6.80,87.18,0.0,0.0,0.0,18.0,14.0,4.0,0.0,3.0,0.0
